Aquí se crearan las variables a partir de los dataset obtenidos en el etl, para preparar los datos de entrada y estos sean adecuados para el modelado de ML que se utilizará

In [1]:
import pandas as pd 
import numpy as np 
import pyarrow as pa 
import pyarrow.parquet as pq 
import ast 
from textblob import TextBlob
import warnings
warnings.filterwarnings('ignore')

Extraemos los archivos generados posterior al etl para trabajar en ellos

In [10]:
reviews = pd.read_parquet(r'D:\Potato\LABS Henry\PI_ML_OPS\ETL-EDA\Archivos\reviews.parquet')

In [11]:
reviews.head(1)

,user_id,user_url,funny,helpful,item_id,last_edited,posted,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,No ratings yet,1250,None,2011-11-05,True,Simple yet with great replayability. In my opi...


## Análisis de sentimientos
Para esta parte se utilizará el DF de user_reviews(reviews.parquet), ya que crearemos una columna llamada 'sentimient_analysis' y utilizaremos la librería de TextBlob. Si la review es negativa se tomará como 0, si es neutral será 1 y positivo será 2

In [12]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59161 entries, 0 to 59160
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59161 non-null  object
 1   user_url     59161 non-null  object
 2   funny        8127 non-null   object
 3   helpful      59161 non-null  object
 4   item_id      59161 non-null  int64 
 5   last_edited  6123 non-null   object
 6   posted       49070 non-null  object
 7   recommend    59161 non-null  bool  
 8   review       59131 non-null  object
dtypes: bool(1), int64(1), object(7)
memory usage: 3.7+ MB


In [13]:
def get_analysis(texto):
    if texto is None:
        return 1 #Neutral
    analisis = TextBlob(texto)
    sentimiento = analisis.sentiment.polarity
    if sentimiento < 0:
        return 0 #negativo
    elif sentimiento > 0:
        return 2 #positivo
    else:
        return 1 #neutral

In [14]:
#se crea la columna 'sentiment_analysis' y se aplica la función para llenarla
reviews['sentiment_analysis'] = reviews['review'].astype(str).apply(get_analysis)
reviews.head(5)

,user_id,user_url,funny,helpful,item_id,last_edited,posted,recommend,review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,No ratings yet,1250,None,2011-11-05,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,No ratings yet,22200,None,2011-07-15,True,It's unique and worth a playthrough.,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,No ratings yet,43110,None,2011-04-21,True,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,http://steamcommunity.com/id/js41637,None,15 of 20 people (75%) found this review helpful,251610,None,2014-06-24,True,I know what you think when you see this title ...,2
4,js41637,http://steamcommunity.com/id/js41637,None,0 of 1 people (0%) found this review helpful,227300,None,2013-09-08,True,For a simple (it's actually not all that simpl...,0


In [15]:
#filtramos una reseña para comprobar que el analisis se ha hecho de manera correcta
reviews['review'][10]

'Fun balance of tactics and strategy.  Potential for very rewarding battles on smaller maps.  Can become a bit of a grind on larger maps (>200 stars).'

In [16]:
reviews['sentiment_analysis'][10]

2

In [9]:
reviews.head(1)

,user_id,user_url,funny,helpful,item_id,last_edited,posted,recommend,review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,No ratings yet,1250,None,2011-11-05,True,Simple yet with great replayability. In my opi...,2


In [17]:
# Eliminamos la columna reviews ya que no será necesaria para su uso y otras columnas que no son necesarias
reviews.drop(columns=['review','funny', 'helpful', 'last_edited','user_url'], axis=1, inplace=True)
reviews

,user_id,item_id,posted,recommend,sentiment_analysis
0,76561197970982479,1250,2011-11-05,True,2
1,76561197970982479,22200,2011-07-15,True,2
2,76561197970982479,43110,2011-04-21,True,2
3,js41637,251610,2014-06-24,True,2
4,js41637,227300,2013-09-08,True,0
...,...,...,...,...,...
59156,Fuckfhaisjnsnsjakaka,70,None,True,2
59157,3214213216,362890,None,True,2
59158,ChrisCoroner,273110,None,True,2
59159,CaptainAmericaCw,730,None,True,2


In [20]:
reviews.to_csv('Archivos/reviews.csv', index=False)


In [21]:
rev = pd.read_csv('Archivos/reviews.csv')

In [23]:
rev.to_parquet('Archivos/reviews_analysis.parquet')

## API
En esta sección crearemos las funcines necesarias para los endpoints

In [11]:
games = pd.read_parquet(r'D:\Potato\LABS Henry\PI_ML_OPS\ETL-EDA\Archivos\steam_parquet.parquet')

In [12]:
games.head(1)

,genres,url,release_date,tags,reviews_url,specs,price,early_access,id,Año,publisher,app_name,title,developer
0,Action,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,['Strategy' 'Action' 'Indie' 'Casual' 'Simulat...,http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro


In [13]:
games.dtypes

genres           object
url              object
release_date     object
tags             object
reviews_url      object
specs            object
price           float64
early_access       bool
id                int64
Año              object
publisher        object
app_name         object
title            object
developer        object
dtype: object

In [14]:
#calculamos las recomendaciones totales del DF
len(reviews['recommend'])

59161

In [15]:
# calculamos los usuarios totales
len(reviews['user_id'].unique())

25467

In [16]:
# comprobamos nulos en el precio 
games['price'].isnull().sum()

0

Aquí se procede a unir las columnas para su uso en la función

In [17]:
# almacenamos la columna price y id para su uso más adelante 
"""price= games[['price','id']]
price = price.drop_duplicates('id',keep='first')
price = price.rename(columns={'id': 'item_id'}) #modificamos el nombre para poder hacer un merge más adelante
price"""

,price,item_id
0,4.99,761140
5,0.00,643980
9,0.00,670290
14,0.99,767400
17,3.99,772540
...,...,...
71535,1.99,745400
71539,1.99,773640
71543,4.99,733530
71546,1.99,610660


In [18]:
items = pd.read_parquet(r'D:\Potato\LABS Henry\PI_ML_OPS\ETL-EDA\Archivos\user_items.parquet')
items.head(1)

,item_id,item_name,playtime_forever,user_id,items_count,steam_id
0,10,Counter-Strike,6,76561197970982479,277,76561197970982479


In [138]:
df_final = games[['genres','price','id','Año','app_name','developer']]
df_final =df_final.rename(columns={'id':'item_id'})
df_final

,genres,price,item_id,Año,app_name,developer
0,Action,4.99,761140,2018,Lost Summoner Kitty,Kotoshiro
1,Casual,4.99,761140,2018,Lost Summoner Kitty,Kotoshiro
2,Indie,4.99,761140,2018,Lost Summoner Kitty,Kotoshiro
3,Simulation,4.99,761140,2018,Lost Summoner Kitty,Kotoshiro
4,Strategy,4.99,761140,2018,Lost Summoner Kitty,Kotoshiro
...,...,...,...,...,...,...
71546,Indie,1.99,610660,2018,Russian Roads,Laush Dmitriy Sergeevich
71547,Racing,1.99,610660,2018,Russian Roads,Laush Dmitriy Sergeevich
71548,Simulation,1.99,610660,2018,Russian Roads,Laush Dmitriy Sergeevich
71549,Casual,4.99,658870,2017,EXIT 2 - Directions,"xropi,stev3ns"


In [136]:
items_df= items[['item_id','playtime_forever','user_id','items_count']]
items_df

,item_id,playtime_forever,user_id,items_count
0,10,6,76561197970982479,277
1,20,0,js41637,888
2,30,7,evcentric,137
3,40,0,Riot-Punch,328
4,50,0,doctr,541
...,...,...,...,...
88171,202090,1501,76561198323066619,22
88172,239220,2374,76561198326700687,177
88173,257730,654,XxLaughingJackClown77xX,0
88174,227220,2973,76561198329548331,7


In [139]:
df_final1 = df_final.merge(items_df,on='item_id')
df_final1.head(5)

,genres,price,item_id,Año,app_name,developer,playtime_forever,user_id,items_count
0,Action,9.99,282010,1997,Carmageddon Max Pack,Stainless Games Ltd,5,76561198063896904,10
1,Action,9.99,282010,1997,Carmageddon Max Pack,Stainless Games Ltd,0,drAlMac,258
2,Action,9.99,282010,1997,Carmageddon Max Pack,Stainless Games Ltd,0,76561198087302899,42
3,Action,9.99,282010,1997,Carmageddon Max Pack,Stainless Games Ltd,0,76561198046358897,33
4,Action,9.99,282010,1997,Carmageddon Max Pack,Stainless Games Ltd,13,76561198091362266,1


In [13]:
df_final1['Año'] = df_final1['Año'].fillna(0)
df_final1['Año']= df_final1['Año'].astype(int)

In [15]:
df_final1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23298157 entries, 0 to 23298157
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   genres              object 
 1   price               float64
 2   item_id             int64  
 3   Año                 int32  
 4   app_name            object 
 5   developer           object 
 6   playtime_forever    int64  
 7   user_id             object 
 8   items_count         int64  
 9   recommend           bool   
 10  sentiment_analysis  int64  
dtypes: bool(1), float64(1), int32(1), int64(4), object(4)
memory usage: 1.8+ GB


In [141]:
reviews_df = reviews[['user_id','item_id','recommend','sentiment_analysis']]
reviews_df.head()

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,76561197970982479,22200,True,2
2,76561197970982479,43110,True,2
3,js41637,251610,True,2
4,js41637,227300,True,0


In [142]:
df_final1= df_final1.merge(reviews_df,on='item_id')
df_final1

In [144]:
df_final1.drop(columns='user_id_y',index= 1,inplace=True)
df_final1= df_final1.rename(columns={'user_id_x': 'user_id'})
df_final1.head(5)

,genres,price,item_id,Año,app_name,developer,playtime_forever,user_id,items_count,recommend,sentiment_analysis
0,Action,9.99,282010,1997,Carmageddon Max Pack,Stainless Games Ltd,5,76561198063896904,10,True,1
2,Action,9.99,282010,1997,Carmageddon Max Pack,Stainless Games Ltd,0,76561198087302899,42,True,1
3,Action,9.99,282010,1997,Carmageddon Max Pack,Stainless Games Ltd,0,76561198046358897,33,True,1
4,Action,9.99,282010,1997,Carmageddon Max Pack,Stainless Games Ltd,13,76561198091362266,1,True,1
5,Indie,9.99,282010,1997,Carmageddon Max Pack,Stainless Games Ltd,5,76561198063896904,10,True,1


Función def **developer**( desarrollador: str)

In [148]:
# la función retorna la cantidad de items y el porcentaje de juegos gratis por año según la desarrolladoras

"""
def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. 
Parametros de entrada: 
    desarrollador del juego steam.Ejemplo: Kotoshiro
Ejemplo de retorno:
    -Año en que se estrenó el juego
    -cantidad de items por año: cantidad de juegos publicados por el desarrollador
    -porcentaje de juegos gratis: 
"""
def developer(desarrollador):

    #filtramos el df desarrollador para compararlo al dato ingresado
    dta =df_final1[df_final1['developer'] == desarrollador]

    #agrupamos por año
    grupo_anio = dta.groupby('Año')['item_id'].count()
    #agrupamos por 'price' paa conocer la cantidad de contenido gratis
    free= dta[dta['price'] == 0.0].groupby('Año')['item_id'].count()
    free_porcentaje= (free/grupo_anio*100).fillna(0).astype(int)

    #creamos un data frame para mostrar cómo salida
    retorno = pd.DataFrame({
        'Año' : grupo_anio.index, #indice
        'Cantidad de items por año': grupo_anio.values, 
        'Porcentaje de juegos gratis': free_porcentaje.values
    })

    return retorno


In [152]:
#verificamos 
dev = 'Valve'
developer(dev)

,Año,Cantidad de items por año,Porcentaje de juegos gratis
0,1998,7874,0
1,1999,1819,0
2,2000,10028,0
3,2001,65,0
4,2003,276,0
5,2004,141749,0
6,2005,3731,100
7,2006,3530,0
8,2007,30028,0
9,2008,3630,0


def **Userdata**(user_id)

In [162]:
"""
def userdata(user_id:str): Pasa como parametro el id de usario a consultar

Retorno: 
        Devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación
        en base a reviews.recommend y cantidad de items.
"""

def userdata(user_id:str): 
    #user =reviews[reviews['user_id']== user_id]
    gastado = df_final1[df_final1['user_id'] == user_id]['price'].sum()

    recomendacion= df_final1[df_final1['user_id'] == user_id]['recommend'].sum()
    total_recom= len(reviews['user_id'].unique())
    porcentaje = (recomendacion/total_recom)*100

    cont= df_final1[df_final1['user_id'] == user_id]['items_count'].iloc[0]

    return{
        'Cantidad de dinero gastado por el usuario': float(gastado),
        'Porcentaje de recomendacion por el usuario': round(float(porcentaje),3),
        'cantidad de items': int(cont)
    }

    
    

In [163]:
user= 'doctr'
userdata(user)

{'Cantidad de dinero gastado por el usuario': 19.96,
 'Porcentaje de recomendacion por el usuario': 0.012,
 'cantidad de items': 541}

Funcion **def userforgenre**(genero:str)

In [164]:
"""
    funcion UserForGenre: devuelve el usuario que acumula más hrs jugadas 
    para el género dado y una lista de la acumulación de hrs jugadas por año
    Ejemplo de salida: 
    {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, 
    "Horas jugadas":[{Año: 2013, Horas: 203}, 
    {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}
"""

def UserforGenre(genero:str):

    try:

        df_genre = df_final1
        filtro = df_genre[df_genre['genres']== genero]
        playtime_sum = filtro.groupby(['user_id','Año'])['playtime_forever'].sum()
        user_maxplay = playtime_sum.groupby('user_id').sum().idxmax()
        play_year = playtime_sum.loc[user_maxplay].to_dict()
        del df_genre, playtime_sum #liberamos espacio   

        return {'Usuario con más horas jugadas para el género '+ genero+ ' es: '+ str(user_maxplay), 
                'Horas jugadas por año: '+ str(play_year)}
    
    except Exception as e:
        return{'Error': str(e)}


In [165]:
gen= 'Simulation'
UserforGenre(gen)

{"Horas jugadas por año: {'2006': 524085782}",
 'Usuario con más horas jugadas para el género Simulation es: 76561198082881423'}

### Funcion def **best_developer_year**(año: int)

In [172]:
#convertimos a tipo entero el año para su uso en la api
df_final1['Año']=df_final1['Año'].replace('sin dato', np.nan)
df_final1['Año'] = pd.to_numeric(df_final1['Año'], errors='coerce', downcast='integer')

In [173]:
df_final1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23298157 entries, 0 to 23298157
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   genres              object 
 1   price               float64
 2   item_id             int64  
 3   Año                 float64
 4   app_name            object 
 5   developer           object 
 6   playtime_forever    int64  
 7   user_id             object 
 8   items_count         int64  
 9   recommend           bool   
 10  sentiment_analysis  int64  
dtypes: bool(1), float64(2), int64(4), object(4)
memory usage: 1.9+ GB


In [176]:
"""
Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por 
usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
"""

def best_developer_year(año:int):
    reviewsxgames = df_final1[['developer', 'Año','recommend','sentiment_analysis']]
    #sentiment_analysis se iguala a 2 porque queremos que sean comentarios unicamente positivos
    filtro = reviewsxgames[(reviewsxgames['Año'] == año) & (reviewsxgames['recommend'] == True) & (reviewsxgames['sentiment_analysis'] == 2)]
    positive_review= filtro['developer'].value_counts()
    top3_pos_dev= positive_review.nlargest(3).index.tolist()
    del reviewsxgames, filtro, positive_review #liberamos espacio
    
    return [{'Puesto 1': top3_pos_dev[0]},{'Puesto 2': top3_pos_dev[1]}, {'Puesto 3': top3_pos_dev[2]}]


In [178]:
top3dev = 2014
best_developer_year(top3dev)

[{'Puesto 1': 'Edge of Reality'},
 {'Puesto 2': 'New World Interactive'},
 {'Puesto 3': 'Endnight Games Ltd'}]

### Función def **developer_review_analysis**(developer)

In [181]:
"""
Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.
Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

"""
def developer_review_analysis(desarrolladora: str):
    reviewxgame = df_final1[['developer','sentiment_analysis']]
    filtro= reviewxgame[reviewxgame['developer'] == desarrolladora]
    sentiment= filtro['sentiment_analysis'].value_counts().to_dict()
    res= {desarrolladora: ['Negative = '+ str(sentiment.get(0,0)),'Neutral = ' +str(sentiment.get(1,0)), 'Positive = ' +str(sentiment.get(2,0))]}
    del reviewxgame, filtro, sentiment #liberamos memoria
    return res

In [182]:
des = 'Valve'
developer_review_analysis(des)

{'Valve': ['Negative = 660945', 'Neutral = 950814', 'Positive = 1891808']}

### Creación de archivos
Guardamos los archivos en formato parquet para su uso en el deploy

In [16]:
df_final1.to_parquet('Archivos/Df_final.parquet') #df con las columnas necesarias para el deploy

